In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings


In [10]:
llm = OllamaLLM(model="llama3.1")
embeddings = OllamaEmbeddings(
    model="llama3.1",
)

In [5]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})


## Prompt Customisation

In [11]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

In [15]:
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("What is Task Decomposition?"), flush=True)

Task Decomposition is the process of breaking down complex tasks into smaller and more manageable steps. This can be done using techniques like Chain of Thought (CoT), Tree of Thoughts, or simple prompting to enable a model or agent to plan ahead.

Thanks for asking!


In [18]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, flush=True, end="")

Task decomposition is the process of breaking down complicated tasks into smaller and simpler steps to facilitate planning and execution. This can be achieved through various methods, such as using language models with simple prompting, task-specific instructions, or even human input. It helps an agent understand what steps are involved in a task and plan ahead accordingly.

thanks for asking!